# Apartment search using ORS and Ohsome API 


In this notebook, we will perform a search for a new apartment in Hamburg based on the surrounding amenities. In addition to popular python packages for geospatial data processing such as `geopandas` we will also use the openrouteservice and ohsome APIs.  

**Execute each cell one by one first** to see how it works by clicking **the `Run` button** in the menu above or **pressing `Shift` + `Enter`** on your keyboard. Afterwards, go through the notebook again and change some of paramters in the cell where it says "**Exercise**" and execute the following cells to run the analysis with different parameters.

## Some required Python Basics

Before we get starte, there are some basics in python that you need to understand. In Python there are variables or objects which can hold different types of values. 

**Numbers** 

In [ ]:
x = 5
y = 4.2

In [ ]:
x

In [ ]:
y

**Characters**

In [ ]:
x = "hello"

In [ ]:
x

**Lists**

In [ ]:
l = [1,2,6]

In [ ]:
l

**Dictionaries** consisting of key-value pairs

In [ ]:
d = {"key": "value"}

In [ ]:
d

In [ ]:
d2 = {"name": "Mona-Lisa", 
     "age": 516}

In [ ]:
d2

**Functions:**  Functions are used to perform operations on the data. 

In [ ]:
x = 5
y = 7
sum([x,y])

Python contains some built-in functions such as `sum()`, but they are limited. You can import additional functions from packages. These are the one that we need for our analysis

In [ ]:
# These are just some imports, which means loading additional functionalities in Python. Just ignore this. 
from ipyleaflet import Map, Marker, GeoData, basemaps,ScaleControl, LayersControl, GeoJSON, LegendControl
from ipywidgets import Layout
import json
import os
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

## Create a list of potential apartments

We store the locations of three potential appartments which we may have found online in a dictionary with geographic coordinates.

**Exercise:** Change the coordinates of the locations or add additinal ones to the dictionary and run the analysis again.  

In [ ]:
apartments = {'0': {'location': [9.981727, 53.552018]}, # longitude, latitude
              '1': {'location': [9.921727, 53.552018]},
              '2': {'location': [9.981727, 53.572018]}}

Let's convert it into a table.

In [ ]:
apartments_df = gpd.GeoDataFrame().from_dict(apartments, orient="index")
apartments_df["geometry"] = apartments_df.location.map(lambda x: Point(x[0], x[1]))

In [ ]:
apartments_df

### Map of apartments

In [ ]:
# Create map with basemap
map_center = list(apartments_df.dissolve().centroid[0].coords)[0]
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))
# Add layer control 
control = LayersControl(position='topright')
m.add_control(control)

display(m)

## Count number of restaurants nearby each appartment

Let's assume it is very important to you that you have many restaurants in walking distance of your apartment. So we will count how many restaurants are within a 500 meter walking distance of each appartment. There are two steps: 

1. Calculate the area within 500 meter walking distance around each appartment using openrouteservice isochrones
2. Downloading and coubnting the restaurants from OpenStreetMap using the ohsome API. 

### 1. Create isochrones around each apartment

First, we will request the isochrones around each apartment and visualize them in the map. For details about the parameters see the [ORS documentation](https://openrouteservice.org/dev/#/api-docs/isochrones). Play around with the settings to find out what each variable is for.

In order to use the OpenRouteSerive API, you need to [create an account and generate a free API key](https://openrouteservice.org/dev/#/login). You can use your github account for it.

In [ ]:
from openrouteservice import client

In [ ]:
ors_api_key = '5b3ce3597851110001cf6248eef794d1244544f7826f417356aee9e4' #Replace this with your own personal ORS API key

In [ ]:
ors_client = client.Client(key=ors_api_key) 

**Execise:** Switch the routing profile to cycling by replacing `foot-walking` with `cycling-regular` and run the cells below again to create new isochrones. 

In [ ]:
ors_settings = {'profile': 'foot-walking', # ORS routing profile
              'intervals': [500], # Interval of isochrones in min if range_type=time
              'segments': 500, 
              'range_type':  'time', #Type of isochones: time or distance
              'attributes': ['area'], # Get area of each isochrone
              'locations' : list(apartments_df.location)
             }

In [ ]:
isochrones = ors_client.isochrones(**ors_settings) 

#### Map with appartments and isochfrom_dict

In [ ]:
# Create map with basemap (available basemaps: https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, basemap=basemaps.CartoDB.Positron)

# Add isochrones 
isochones_layer = GeoJSON(
    data=isochrones,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    })
m.add_layer(isochones_layer)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))

control = LayersControl(position='topright')
m.add_control(control)

display(m)

## 2. Number of restaurants nearby apartments using Ohsome API
Now that we created the isochrones, we want to know how many restaurants there are in the neighbourhood. For this purpose, we will use the Ohsome API. 

In [ ]:
import ohsome
ohsome_client = ohsome.OhsomeClient()

### 2.1. Download restaurants as features to visualized them on the map

**Exercise:** Set a different tag in the the `osm_filter`. 

In [ ]:
osm_filer = "amenity=restaurant"

In [ ]:
restaurant_features = ohsome_client.elements.centroid.post(bpolys=json.dumps(isochrones),
                                                           properties="tags",
                                                             filter=osm_filer)
restaurants_df = restaurant_features.as_dataframe()

In [ ]:
restaurants_df.head()

### 2.2. Intersect isochrones with restaurants and count them

We convert the isochrones into a dataframe called `isochrones_df` and intersect it with the dataframe `restaurants_df` containing the restaurants.

In [ ]:
isochones_df = gpd.GeoDataFrame.from_features(isochrones['features'], crs="epsg:4326")
isochrone_restaurants = gpd.sjoin(restaurants_df, isochones_df, how="left").groupby("group_index").count()
apartments_df["restaurants"] = list(isochrone_restaurants["amenity"])

In [ ]:
apartments_df

### Alternative:  Count number of restaurants within isochrones using ohsome API

Alternatively, we could have calculated the number of restauratans within in each isochrone also using the ohsome API directly without downloading the restaurant features. 

In [ ]:
response = ohsome_client.elements.count.groupByBoundary.post(bpolys=json.dumps(isochrones),
                                                             filter="amenity=restaurant")
restaurants = response.as_dataframe()["value"]
apartments_df["restaurants_ohsome"] = list(restaurants)

In [ ]:
apartments_df

### Map with apartments and restaurants

**Exercise:** You may adapt the style of the final map by change some of the parameters, e.g. the color or radius of the points in the dictionary, e.g. set `'radius':5`. You may also change the basemap by replacing the value of `basemap=basemaps.CartoDB.Positron` in the first two lines e.g. set it to `basemap=basemaps.OpenStreetMap.Mapnik`. Refer to the documentation for [available basemaps](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html).

In [ ]:
# Create map with basemap (available basemaps: https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)
m = Map(center=map_center[::-1], zoom=13, scroll_wheel_zoom=True, layout=Layout(width='100%', height='500px'),
        basemap=basemaps.CartoDB.Positron)

# Add isochrones 
isochones_layer = GeoJSON(
    data=isochrones,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1, 'fillColor': '#c5e8ef'
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    })
m.add_layer(isochones_layer)

# Add appartments as points
apartment_layer = GeoData(geo_dataframe = apartments_df,
    style={'color': 'black', 
           'radius':8, 
           'fillColor': '#3366cc', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Apartments')
m.add_layer(apartment_layer)

# Add restaurants as points
restaurants_layer = GeoData(geo_dataframe = restaurants_df,
    style={'color': 'black', 
           'radius':4, 
           'fillColor': 'red', 
           'opacity':0.5, 
           'weight':1.9, 
           'dashArray':'2', 
           'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 
                 'fillOpacity': 0.2},
    point_style={'radius': 5, 
                 'color': 'red', 
                 'fillOpacity': 0.8, 
                 'fillColor': 'blue', 
                 'weight': 3},
    name = 'Restaurants')

m.add_layer(restaurants_layer)

# Add a scale to map 
m.add_control(ScaleControl(position='bottomleft'))

control = LayersControl(position='topright')
m.add_control(control)

# Add legend
legend = LegendControl({"Apartments":"#3366cc", 
                        "Restaurants":"red", 
                        "Isochrones [500m]":"#c5e8ef"}, name="Legend", position="bottomright")
m.add_control(legend)

display(m)